In [ ]:
! pip install -U kaleido
! pip install gradio
! pip install transformers

In [ ]:
! git clone https://github.com/LC1332/Luotuo-Text-Embedding

In [ ]:
% cd Luotuo-Text-Embedding/gradio

In [1]:
import gradio as gr
from scipy.spatial.distance import cosine
from transformers import AutoModel, AutoTokenizer
from argparse import Namespace
import torch
# Import our models. The package will take care of downloading the models automatically

/home/luotuo/miniconda3/envs/gradio/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from divide import divide_inputs,get_all_embedding
from tsne import TSNE_Plot
tokenizer = AutoTokenizer.from_pretrained("silk-road/luotuo-bert")
model_args = Namespace(do_mlm=None,
                       pooler_type="cls",
                       temp=0.05,
                       mlp_only_train=False,
                       init_embeddings_model=None)
model = AutoModel.from_pretrained("silk-road/luotuo-bert",
                                  trust_remote_code=True,
                                  model_args=model_args)

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


In [3]:
def generate_image(text_input):
    # 将输入的文本按行分割并保存到列表中
    text_left, text_right = divide_inputs(text_input)
    embeddings_left, embeddings_right = get_all_embedding(
        model, tokenizer, text_left, text_right)
    # 在这里编写您自己的生成图片代码，您可以根据需要修改此函数
    # 示例：使用 PIL 创建一个包含文本的简单图片
    merged_list = text_left + text_right
    merged_embed = torch.cat((embeddings_left, embeddings_right), dim=0)
    tsne_plot = TSNE_Plot(merged_list, merged_embed, n_clusters=4)
    fig = tsne_plot.tsne_plot(n_sentence=40)
    return fig

In [15]:
# 创建 Gradio 界面
iface = gr.Interface(fn=generate_image,
                     inputs=gr.inputs.Textbox(lines=50,
                                              placeholder='在此输入文本，一行一句...'),
                     outputs=gr.outputs.Image('pil'),
                     layout="vertical")


iface.launch(share = True,debug=True)

/home/luotuo/miniconda3/envs/gradio/lib/python3.9/site-packages/gradio/deprecation.py:40: UserWarning:

`layout` parameter is deprecated, and it has no effect



Running on local URL:  http://127.0.0.1:7860
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Running on public URL: https://f31f9c0899c0d9662e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


/home/luotuo/miniconda3/envs/gradio/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://f31f9c0899c0d9662e.gradio.live
